#### 107年01月~108年02月 0050 元大台灣50 各日成交資訊:
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20180101&stockNo=0050
#### We need a list of date string in the date range we want.

In [13]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

start_date = "2018-01-01"
stop_date = "2019-02-01"

start = datetime.strptime(start_date, "%Y-%m-%d")
stop = datetime.strptime(stop_date, "%Y-%m-%d")

dates = list()
while start <= stop:
    dates.append(start.strftime('%Y%m%d'))
    start = start + relativedelta(months=1) #每次增加一個月
print(dates)

['20180101', '20180201', '20180301', '20180401', '20180501', '20180601', '20180701', '20180801', '20180901', '20181001', '20181101', '20181201', '20190101', '20190201']


#### Now write a function to parse the HTML response, return the data we want(title, content, ...etc).

In [14]:
import requests
from bs4 import BeautifulSoup as bs

In [17]:
def process_document(document, date):
    
    nodes = document.select('tbody > tr')
    data = list()

    for tr in nodes:
        nodes1 = tr.select('td')
         # append new row
        data.append({
            '日期' : nodes1[0].get_text(),
            '成交股數': nodes1[1].get_text(),
            '成交金額' : nodes1[2].get_text(),
            '開盤價' : nodes1[3].get_text(),
            '最高價' : nodes1[4].get_text(),
            '最低價' : nodes1[5].get_text(),
            '收盤價' : nodes1[6].get_text(),
            '漲跌價差' : nodes1[7].get_text(),
            '成交筆數' : nodes1[8].get_text(),
            
        })
            
        
    return data

#### Crawl over the news on the site, store the data in variable "all_data" .

In [18]:
cnt = 0
all_data = list()
for date in dates:
    print('start crawling :', date)
    res = requests.get('http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date='+date+'&stockNo=0050')
    doc = bs(res.text, 'lxml')
    data = process_document(doc, date)
    all_data += data

start crawling : 20180101
start crawling : 20180201
start crawling : 20180301
start crawling : 20180401
start crawling : 20180501
start crawling : 20180601
start crawling : 20180701
start crawling : 20180801
start crawling : 20180901
start crawling : 20181001
start crawling : 20181101
start crawling : 20181201
start crawling : 20190101
start crawling : 20190201


#### Check the result

In [19]:
all_data[0:5]

[{'成交筆數': '1,795',
  '成交股數': '3,613,199',
  '成交金額': '298,033,251',
  '收盤價': '82.60',
  '日期': '107/01/02',
  '最低價': '82.15',
  '最高價': '82.60',
  '漲跌價差': '+0.45',
  '開盤價': '82.15'},
 {'成交筆數': '3,014',
  '成交股數': '7,196,437',
  '成交金額': '599,476,413',
  '收盤價': '83.35',
  '日期': '107/01/03',
  '最低價': '82.95',
  '最高價': '83.45',
  '漲跌價差': '+0.75',
  '開盤價': '82.95'},
 {'成交筆數': '2,278',
  '成交股數': '5,627,565',
  '成交金額': '469,312,820',
  '收盤價': '83.50',
  '日期': '107/01/04',
  '最低價': '83.20',
  '最高價': '83.65',
  '漲跌價差': '+0.15',
  '開盤價': '83.50'},
 {'成交筆數': '2,562',
  '成交股數': '7,616,796',
  '成交金額': '636,768,692',
  '收盤價': '83.75',
  '日期': '107/01/05',
  '最低價': '83.30',
  '最高價': '83.80',
  '漲跌價差': '+0.25',
  '開盤價': '83.50'},
 {'成交筆數': '2,282',
  '成交股數': '5,189,611',
  '成交金額': '435,775,639',
  '收盤價': '84.10',
  '日期': '107/01/08',
  '最低價': '83.70',
  '最高價': '84.15',
  '漲跌價差': '+0.35',
  '開盤價': '83.80'}]

#### Save as pkl file

In [20]:
import pickle

with open('data/stock.pkl', 'wb') as f:
    pickle.dump(all_data, f)

#### Turn it into pandas dataframe

In [24]:
import pandas as pd
ETF50 = pd.DataFrame(all_data)[['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']]
ETF50.set_index("日期" , inplace=True)
ETF50

,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
日期,,,,,,,,
107/01/02,"3,613,199","298,033,251",82.15,82.60,82.15,82.60,+0.45,"1,795"
107/01/03,"7,196,437","599,476,413",82.95,83.45,82.95,83.35,+0.75,"3,014"
107/01/04,"5,627,565","469,312,820",83.50,83.65,83.20,83.50,+0.15,"2,278"
107/01/05,"7,616,796","636,768,692",83.50,83.80,83.30,83.75,+0.25,"2,562"
107/01/08,"5,189,611","435,775,639",83.80,84.15,83.70,84.10,+0.35,"2,282"
107/01/09,"3,664,716","308,057,554",84.10,84.25,83.90,84.15,+0.05,"1,340"
107/01/10,"5,749,597","482,358,793",84.10,84.25,83.55,83.75,-0.40,"1,073"
107/01/11,"2,579,225","214,867,793",83.65,83.70,83.10,83.40,-0.35,735
107/01/12,"2,871,193","241,214,778",83.55,84.20,83.50,84.10,+0.70,"1,002"
